### **两个明显问题** : ###
***1. 如何在标签噪声下提高文本表示学习的准确性***
<BR>
***2. 如何保证深度神经网络模型的可扩展性, 以有效处理有大量标签噪声的大数据集***

$\ GB_{RAIN}\ $由三部分组成 : 
1. 标记的数据输入文本编码器 , 映射为高位特征向量
2. 粒球聚类层适应性地对相似特征向量进行聚类，形成有相似上下文的特征球，并生成每个特征球的中心向量和标签
3. 将所有粒球的中心向量和标记视为粗表示的实例和标记，用于训练神经网络
<BR>
### **主要贡献** ###
1. 提出了一种有效的高维向量颗粒球聚类算法，该算法可以嵌入到不同的深度神经网络模型中，在保持数据特征的前提下对相似文本进行聚类
2. 提出了一个基于粗粒度表示的鲁棒训练框架，以帮助$\ DNN\ $模型对抗$\ NLP\ $中的标签噪声
3. 对流行的NLP任务的广泛评估表明，比现有的工作有更好的鲁棒性


**输入** : 带标签噪声的训练集
<BR><BR>
每个样本是一个带（不一定正确）标签的句子 ，输入文本由编码器（一个表示网络）处理 ， 映射成高维表示空间的特征向量 ， 然后粒球计算层基于特征向量把相近的输入进行聚类 ， 构建多粒度语料库（ 即粒球 ）。所有样本基于粒球重新表示其表现和标签 ， 保留数据特征同时纠正标签噪声 。 重新表示的实例输入分类器 ， 根据输出概率分布来预测标签 。 根据预测和重新表示的标签计算损失值 ， 损失值反映了模型在训练数据上的性能。

### **用粒球重新表示输入** ###
用粗粒度表示学习重新表示输入

### **粒球定义** ###
$\text{对于一批样本}\ \mathcal{D}\ =\ \left\{\ (\ x_1\ ,\ y_1\ )\ ,\ (\ x_2\ ,\ y_2\ )\ ,\ \dots\ ,\ (\ x_{sm}\ ,\ y_{sm}\ )\ \right\}\ ,\ x_i\ \text{为样本特征向量}\ ,\ y_i\ \text{为样本标签}\ ,\ $
<BR>
$$
Granular Ball\ =\ \left\{\ (\ x_{s1}\ ,\ y_{s1}\ )\ ,\ (\ x_{s2}\ ,\ y_{s2}\ )\ ,\ \dots\ ,\ (\ x_{sm}\ ,\ y_{sm}\ )\ \right\}\\ 
$$
$$
Y_{GB}\ =\ \arg\underset{p}{\max}\left(\ \frac{count(\ y_{si})\ =\ p}{m}\right)
$$
$$
C_{GB}\ =\ \frac{1}{N}\overset{N}{\underset{i=1}{\sum}}x_{si}
$$
$\text{其中}\ ,\ (\ x_{si}\ ,\ y_{si}\ )\ \in\ \mathcal{D}\ \text{也就是说}\ ,\ Granular Ball \subseteq\ \mathcal{D}\ ,\ m\ \text{为粒球大小}\ ,\ \text{所有特征向量}\ x_{si}\text{都接近一个预定义的距离}\ ,\ \text{当前粒球的中心向量}\ C_{GB}\ ,\ \text{标签}\ Y_{GB}$

### **粒球纯度定义** ###
给定一个由$\ k\ $类样本构成的粒球$ GB \ (\ $即对于$\ GB\ $中第$\ i\ $类样本$\ c_i\ ,\ \overset{k}{\underset{i=1}{\bigcup}}c_i\ =\ GB\ ,\ \overset{k}{\underset{i=1}{\bigcap}}c_i\ =\ \phi$)$\ ,\ $纯度
$$
T_{GB}\ =\ \frac{\max|\ c_i\ |}{|\ GB\ |}\ ,\ i\ =\ 1\ ,\ 2\ ,\ \dots\ ,\ k
$$
|.|代表集合中样本的数量

### **高维向量距离定义** ###
给定两高维向量$\ x(\ x_1\ ,\ x_2\ ,\ \dots\ ,\ x_n\ )$和$\ x(\ y_1\ ,\ y_2\ ,\ \dots\ ,\ y_n\ )\ ,\ $两向量的距离 : 
$$
d\ =\ x\ -\ y\ =\ \left(\ (\ x_1\ -\ y_1\ )\ ,\ (\ x_2\ -\ y_2\ )\ ,\ \dots\ ,\ (\ x_n\ -\ y_n\ )\ \right)
$$
$$
Dis(\ x\ ,\ y\ )\ =\ ||\ (\ α_1\ ,\ α_2\ ,\ \dots\ ,\ α_n)d^T\ ||_2
$$
**d** 是一个用于保持 x 和 y 的区别的 n 维向量 ， Dis() : 加权距离 , $α_i\ :\ $不同权值的超参数 , 如果$\ d_i \geq\ d_j\ ,\ $则$\ α_i \ \geq\ α_j\ $

由于分到粒球后 , 一个粒球中所有样本相近 , 语义相似 , 故用球中样本最多的标签作为球中所有样本和球的标签 。 因此纠正标签噪声

### **$\ GB_{RAIN}\ $的聚类算法** ###
**数据 :**
$D\ :\ $一批样本$\qquad T\ :\ $纯度的阈值
<BR>
**结果 :**
$GB_s : $所有覆盖了整个输入数据的粒球 , 连同的中心向量和标签
<BR>
<img src=./img/img1.png alt="img1" width=600><img src=./img/img2.png alt="img2" width=600>

### **优化** ###
反向传播阶段 , 基于卷积神经网络(CNN)中平均池化的反向传播，设计了一种确保梯度信息从分类器有效传输到每个粒球的方法 。
对于属于粒球$\ gb_j\ $的样本$\ ,\ $它们的梯度由$\ gb_j\ $的中心向量$\ c_j\ $的梯度损失决定$\ ,\ $确保了反向传播中梯度从粒球传播到单个样本$\ ,\ $最大区别在于不使用粒球平均梯度$\ ,\ $而是应用加权策略$\ ,\ $根据粒球中样本数量分配不同权重$\ \gamma\ ,\ $设置了阈值$\ ,\ $目前为3$\ ,\ $当粒球中样本数少于$\ 3\ $时$\ ,\ \gamma\ $设为$\ 0\ ,\ $否则为$\ 1\ $

给定输入向量$\ x_i\ ,\ $和一个有中心向量$\ c_j\ $的粒球$\ gb_j\ ,\ $关于$\ x_i\ $的梯度损失函数$\ L\ $计算为 :
$$
\frac{\partial L}{\partial x_i}\ =\ \gamma\ \frac{\partial L}{\partial c_j}\ ,\ for\ x_j\ \in\ gb_j\ ,\ \gamma\ =\ (\ 0||1\ )
$$


### **在$\ Text\ Classification\ $上的实验** ###
**数据集 :** AGNews 和 IMDb
<BR>
**模型 :** 将$\ GB_{RAIN}\ $用于三个不同的文本分类模型$\ BiLSTM\ ,\ GRU\ $和$\ Text-CNN\ ,\ $得到了$\ GB-BiLSTM\ ,\ GB-GRU\ $和$\ GB-Text-CNN\ ,\ $因此有六种模型$\ ,\ $前二者用有$\ 128\ $个隐藏节点的双层网络$\ ,\ $后者用三个卷积核大小为$\ 3\ ,4\ ,5\ $的卷积层$\ ,\ $单词向量维数初始化为$\ 100\ $
<BR>
**环境 :** 学习率0.001$\ ,\ $在两个NVIDIA GeForce RTX 3090 gpu上运行。